
# 漢字・アルファベットからカタカナを推測するモデルを学習する
オープンソース辞書及び，Wikipediaから作成した漢字とカタカナのペアデータを用いて，Transformer+Seq2Seqのモデルで，スクラッチで学習する。

入力の文字（漢字・アルファベットなど）は一文字ずつに分割，また，出力の文字（カタカナ）も一文字ずつに分割して，学習する。



## データ準備
データは，[dict](../dict)で作成した姓名辞書データのoss.json及びseimei.jsonと，Wikipediaから抜き出した，漢字姓名とカタカナ姓名（スペース区切りのもの）を用いて，学習に使用した。単語単位である姓名辞書データと，Wikipediaのスペースで区切られた姓名を用いることで，姓名で漢字とカナが入力された場合の学習を可能とした。

また，ひらがなとカタカナ，


In [1]:
# スクリプトで使う環境変数をセット
import os
os.environ['DIR']='dataset'
os.environ['MODEL']='model'
os.environ["LOG"]='logs'
os.environ['RATIO']="0.01"
os.environ['WIKI']="../dict/wikipedia/wikiname.txt"

os.makedirs(os.environ['DIR'],exist_ok=True)
os.makedirs(os.environ['MODEL'],exist_ok=True)

In [2]:
# オープンソース辞書とWikipediaから作成した単語辞書をマージする
!python mergedic.py --dic1 ../dict/oss.json --dic2 ../dict/seimei.json --outfile tmp.json



{
  "dic1": "../dict/oss.json",
  "dic2": "../dict/seimei.json",
  "outfile": "tmp.json"
}
len=1620498
cnt=1673683


In [3]:
# 作成したデータを訓練用，開発用，検証用に分ける。開発用と検証用がそれぞれ全体の0.01
!python prep.py --json tmp.json --outdir $DIR --ratio $RATIO


{
  "json": "tmp.json",
  "outdir": "dataset",
  "ratio": 0.01
}
k_max_len=72
v_max_len=120
train,1640211
valid,16736
test,16736


In [4]:
# wikipediaの漢字姓名とカナ姓名を取得し，訓練，開発，検証用に分ける。
!python catawk.py --begin_idx_per 0 --end_idx_per 1 --appendfile $DIR/valid.src --index 0 --infile $WIKI --infile_delimiter tsv
!python catawk.py --begin_idx_per 0 --end_idx_per 1 --appendfile $DIR/valid.tgt --index 1 --infile $WIKI --infile_delimiter tsv
!python catawk.py --begin_idx_per 1 --end_idx_per 2 --appendfile $DIR/test.src --index 0 --infile $WIKI --infile_delimiter tsv
!python catawk.py --begin_idx_per 1 --end_idx_per 2 --appendfile $DIR/test.tgt --index 1 --infile $WIKI --infile_delimiter tsv
!python catawk.py --begin_idx_per 2 --end_idx_per 100 --appendfile $DIR/train.src --index 0 --infile $WIKI --infile_delimiter tsv
!python catawk.py --begin_idx_per 2 --end_idx_per 100 --appendfile $DIR/train.tgt --index 1 --infile $WIKI --infile_delimiter tsv

In [5]:
# ひらがなとカタカナのペアデータを作成し，訓練データに混ぜる
!python kana.py --outfile $DIR/hirakata.txt

!python catawk.py --begin_idx_per 0 --end_idx_per 100 --appendfile $DIR/train.src --index 0 --infile $DIR/hirakata.txt
!python catawk.py --begin_idx_per 0 --end_idx_per 100 --appendfile $DIR/train.tgt --index 1 --infile $DIR/hirakata.txt




{
  "outfile": "dataset/hirakata.txt"
}


In [6]:
# 単漢字とカタカナのデータを，訓練データに混ぜる 
!python jsontotext.py --jsonfile ../dict/tankanji.json --outfile tankanji.txt

!python catawk.py --begin_idx_per 0 --end_idx_per 100 --appendfile $DIR/train.src --index 0 --infile tankanji.txt
!python catawk.py --begin_idx_per 0 --end_idx_per 100 --appendfile $DIR/train.tgt --index 1 --infile tankanji.txt

{
  "jsonfile": "../dict/tankanji.json",
  "outfile": "tankanji.txt"
}


In [7]:
# 開発，検証用に，訓練データに入っているものがあれば，削除する(インサンプルを阻止)
!python omit_testval.py --train_src $DIR/train.src --train_tgt $DIR/train.tgt --test_src $DIR/test.src --test_tgt $DIR/test.tgt --valid_src $DIR/valid.src --valid_tgt $DIR/valid.tgt



{
  "train_src": "dataset/train.src",
  "train_tgt": "dataset/train.tgt",
  "test_src": "dataset/test.src",
  "test_tgt": "dataset/test.tgt",
  "valid_src": "dataset/valid.src",
  "valid_tgt": "dataset/valid.tgt"
}
train=1924400
omit=508


In [8]:
# 訓練，開発，検証データをシャッフルする

!python shuffle.py --src $DIR/train.src --tgt $DIR/train.tgt
!python shuffle.py --src $DIR/valid.src --tgt $DIR/valid.tgt
!python shuffle.py --src $DIR/test.src --tgt $DIR/test.tgt



{
  "tgt": "dataset/train.tgt",
  "src": "dataset/train.src"
}
{
  "tgt": "dataset/valid.tgt",
  "src": "dataset/valid.src"
}
{
  "tgt": "dataset/test.tgt",
  "src": "dataset/test.src"
}


In [9]:
# データをスペース区切りにする（文字単位で，学習させるため）

!python space.py --infile $DIR/train.src --outfile $DIR/train.src
!python space.py --infile $DIR/train.tgt --outfile $DIR/train.tgt
!python space.py --infile $DIR/valid.src --outfile $DIR/valid.src
!python space.py --infile $DIR/valid.tgt --outfile $DIR/valid.tgt
!python space.py --infile $DIR/test.src --outfile $DIR/test.src
!python space.py --infile $DIR/test.tgt --outfile $DIR/test.tgt



{
  "infile": "dataset/train.src",
  "outfile": "dataset/train.src"
}
{
  "infile": "dataset/train.tgt",
  "outfile": "dataset/train.tgt"
}
{
  "infile": "dataset/valid.src",
  "outfile": "dataset/valid.src"
}
{
  "infile": "dataset/valid.tgt",
  "outfile": "dataset/valid.tgt"
}
{
  "infile": "dataset/test.src",
  "outfile": "dataset/test.src"
}
{
  "infile": "dataset/test.tgt",
  "outfile": "dataset/test.tgt"
}


In [10]:
# jsonl形式に変換する

!python format.py --src $DIR/train.src --tgt $DIR/train.tgt --outfile $DIR/train.jsonl
!python format.py --src $DIR/test.src --tgt $DIR/test.tgt --outfile $DIR/test.jsonl
!python format.py --src $DIR/valid.src --tgt $DIR/valid.tgt --outfile $DIR/valid.jsonl

## 漢字・アルファベットとカタカナのペアから学習する
学習は，訓練用データを用いて学習し，１エポックごとに開発用データ(valid.json)でLossを確認する。


### パラメタ
- emb_size 
入力，出力の文字のエンベッドのDimension

- nhead
マルチヘッド数

- ffn_hid_dim
FFNのDimension

- num_encoder_layer
エンコードレイヤーの数

- num_decoder_layer
デコードレイヤーの数

- lr 
学習率

- dropout
ドロップアウトの割合, 0-1

- num_epochs
何周学習データを用いて学習を行うか

- device
mps,cpu,cudaから選択
-- mps
アップルシリコンを搭載したマシンで実行する際に選択
-- cuda
CUDAが利用できる環境で選択
-- cpu
上記以外は，CPUモードを選択

- earlystop_patient
開発用データでlossが下がらなくなってearlystop_patient回計算が終了した場合に，num_epochs数以下でも，計算を終了させる

- output_dir
モデルの出力ディレクトリ

- tensorboard_logdir
tensorboard形式のログの出力ディレクトリ。学習状況を確認するためには`tensorboard --logdir logs`を実行後，ブラウザでhttp://localhost:6000/から確認
 



- prefix
jsonl形式のデータのprefix

- source_lang
jsonl形式のデータのsourceのキー

- target_lang
jsonl形式のデータのtargetのキー

- train_file
訓練用データのJSONLファイル

- valid_file
開発用データのJSONLファイル

In [11]:
# 訓練実行

!python ./transformer_model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 32 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 50 \
  --device cuda \
  --earlystop_patient 3 \
  --output_dir $MODEL \
  --tensorboard_logdir $LOG \
  --prefix translation \
  --source_lang kanji \
  --target_lang kana \
  --train_file $DIR/train.jsonl \
  --valid_file $DIR/valid.jsonl


/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
num_epochs:50
/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/venv/lib/python3.12/site-packages/torch/nn/functional.py:5849: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
^C
Traceback (most recent call last):
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/train/./transformer_model.py", line 509, in <module>
    main()
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/train/./transformer_model.py", line 505, in main
    KanjiKanaTransformer(args).train()
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/train/./transformer

## モデルを検証する
作成したモデルを，検証用データを用いて検証し，正解率を算出する


## パラメタ

- test_file
jsonl形式の検証用データファイル

- model_file
transformer_model.pyで作成したモデルファイル。

- outfile
検証結果を格納するファイル

- device
cpu限定

-nbest
検証用データの漢字姓名を入力データとした時に，モデルが出力するカタカナ姓名を確率の高い方からnbest個出力する。beam_width以下。searchパラメタがbothかbeamの時有効

- beam_width
ビームサーチのビーム幅。searchパラメタがbothかbeamの時有効

- max_len
出力するカタカナ姓名の最大長さ

- search
greedy,beam,bothから選択
-- greedy
貪欲法による探索
-- beam
ビームサーチによる探索
-- both
貪欲砲，ビームサーチ両方の探索



In [ ]:
# 検証用データを実行


!python generate_batch.py \
  --test_file $DIR/test.jsonl \
  --model_file $MODEL/checkpoint_best.pt \
  --outfile $DIR/generate.txt \
  --device cpu \
  --nbest 5 \
  --beam_width 5 \
  --max_len 100 \
  --search both





In [ ]:
# 検証結果を評価
!python evaluate.py --infile $DIR/generate.txt



# カタカナから漢字を推測するモデルを学習する
漢字からカタカナを推測するモデルと同様に，オープンソース辞書及び，Wikipediaから作成した漢字とカタカナのペアデータを用いて，Transformer+Seq2Seqのモデルで，スクラッチで学習する。

入力の文字（カタカナ）は一文字ずつに分割，また，出力の文字（漢字・アルファベットなど）も一文字ずつに分割して，学習する。

データセットは，漢字からカタカナを推測するモデルのデータを，漢字とカナを逆にした者を作成して利用する。



In [ ]:

# 環境変数とディレクトリを作成，及びデータのコピー
import os
import shutil
os.environ['DIR_R']='dataset_r'
os.environ['MODEL_R']='model_r'
os.environ['LOG_R']='logs_r'

os.makedirs(os.environ['DIR_R'],exist_ok=True)
os.makedirs(os.environ['MODEL_R'],exist_ok=True)

# 漢字からカナ用に作成したデータを，SRCとTGTを逆にコピーする
shutil.copy(os.environ['DIR']+"/train.src",os.environ['DIR_R']+"/train.tgt")
shutil.copy(os.environ['DIR']+"/train.tgt",os.environ['DIR_R']+"/train.src")
shutil.copy(os.environ['DIR']+"/valid.src",os.environ['DIR_R']+"/valid.tgt")
shutil.copy(os.environ['DIR']+"/valid.tgt",os.environ['DIR_R']+"/valid.src")
shutil.copy(os.environ['DIR']+"/test.src",os.environ['DIR_R']+"/test.tgt")
shutil.copy(os.environ['DIR']+"/test.tgt",os.environ['DIR_R']+"/test.src")

In [15]:
# jsonl形式に変換する

!python format.py --src $DIR_R/train.src --tgt $DIR_R/train.tgt --outfile $DIR_R/train.jsonl
!python format.py --src $DIR_R/test.src --tgt $DIR_R/test.tgt --outfile $DIR_R/test.jsonl
!python format.py --src $DIR_R/valid.src --tgt $DIR_R/valid.tgt --outfile $DIR_R/valid.jsonl

## 漢字・アルファベットとカタカナのペアから学習する
学習は，訓練用データを用いて学習し，１エポックごとに開発用データ(valid.json)でLossを確認する。


In [ ]:
!python ./transformer_model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 32 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 50 \
  --device mps \
  --earlystop_patient 3 \
  --output_dir $MODEL_R \
  --tensorboard_logdir $LOG_R \
  --prefix translation \
  --source_lang kanji \
  --target_lang kana \
  --train_file $DIR_R/train.jsonl \
  --valid_file $DIR_R/valid.jsonl


## モデルを検証する
作成したモデルを，検証用データを用いて検証し，正解率を算出する

In [ ]:
# 検証用データを実行


!python generate_batch.py \
  --test_file $DIR_R/test.jsonl \
  --model_file $MODEL_R/checkpoint_best.pt \
  --outfile $DIR_R/generate.txt \
  --device cpu \
  --nbest 5 \
  --beam_width 5 \
  --max_len 100 \
  --search both



In [ ]:
# 検証結果を評価
!python evaluate.py --infile $DIR_R/generate.txt

# モデルをJavaのDJL用に変換する
Pytorchで学習したモデルをJavaで使えるようにモデルの変換を行う。



In [ ]:
# 漢字からカナ
!python convert_jitscript.py  \
    --model_file=$MODEL/checkpoint_best.pt \
    --model_script=$MODEL/script.pt \
    --encoder=$MODEL/encoder.pt \
    --decoder=$MODEL/decoder.pt \
    --positional_encoding=$MODEL/positional_encoding.pt \
    --generator=$MODEL/generator.pt \
    --src_tok_emb=$MODEL/src_tok_emb.pt \
    --tgt_tok_emb=$MODEL/tgt_tok_emb.pt \
    --vocab_src=$MODEL/vocab_src.txt \
    --vocab_tgt=$MODEL/vocab_tgt.txt \
    --params=$MODEL/params.json \
    --device=cpu



In [ ]:
# カナから漢字
!python convert_jitscript.py  \
    --model_file=$MODEL_R/checkpoint_best.pt \
    --model_script=$MODEL_R/script.pt \
    --encoder=$MODEL_R/encoder.pt \
    --decoder=$MODEL_R/decoder.pt \
    --positional_encoding=$MODEL_R/positional_encoding.pt \
    --generator=$MODEL_R/generator.pt \
    --src_tok_emb=$MODEL_R/src_tok_emb.pt \
    --tgt_tok_emb=$MODEL_R/tgt_tok_emb.pt \
    --vocab_src=$MODEL_R/vocab_src.txt \
    --vocab_tgt=$MODEL_R/vocab_tgt.txt \
    --params=$MODEL_R/params.json \
    --device=cpu

